In [21]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

In [22]:
# Load the historical weather data
historical_data = pd.read_csv('./Data/historical_weather_cleaned_filtered.csv')

# Load the submission key
submission_key = pd.read_csv('./Data/submission_key.csv')

In [23]:
# Pivot the historical data to get cities as columns and dates as rows
avg_temp_df = historical_data.pivot(index='date', columns='city_id', values='avg_temp_c')

# Convert the index to datetime
avg_temp_df.index = pd.to_datetime(avg_temp_df.index)

In [24]:
# Train an ARIMA model for each city using the historical data up to the end of 2018
predictions = {}

for city in avg_temp_df.columns:
    # Train the ARIMA model on data up to the end of 2018
    train_data = avg_temp_df[avg_temp_df.index < '2019-01-01'][city]
    
    # Fit the model
    model = ARIMA(train_data, order=(5, 1, 0))  # The order (5, 1, 0) can be adjusted based on performance
    model_fit = model.fit()
    
    # Forecast the average temperature for the first week of 2019
    forecast = model_fit.forecast(steps=7)
    
    # Store the forecast in the predictions dictionary
    predictions[city] = forecast.values

c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is ava

In [25]:
# Create a DataFrame for the submission
submission = submission_key.copy()
submission['avg_temp_c'] = submission.apply(lambda row: predictions[row['city_id']][(pd.to_datetime(row['date']) - pd.to_datetime('2019-01-01')).days], axis=1)

submission.head()

,submission_ID,city_id,date,avg_temp_c
0,1,C001,2019-01-01,11.548374
1,2,C001,2019-01-02,11.087903
2,3,C001,2019-01-03,10.936680
3,4,C001,2019-01-04,10.840372
4,5,C001,2019-01-05,10.863584


In [26]:
# Use only submission_ID,avg_temp_c columns
submission = submission[['submission_ID', 'avg_temp_c']]

# Save the submission to a CSV file
submission.to_csv('./Data/submission_ARIMA.csv', index=False)

print("Predicted temperatures for submission:")
print(submission)

Predicted temperatures for submission:
     submission_ID  avg_temp_c
0                1   11.548374
1                2   11.087903
2                3   10.936680
3                4   10.840372
4                5   10.863584
..             ...         ...
695            696   21.307010
696            697   21.402088
697            698   21.227765
698            699   21.124234
699            700   21.132516

[700 rows x 2 columns]


In [27]:
# Print the average temperature data for each city for each day for last day of 2018
print("Last week of 2018:")
avg_temp_df[avg_temp_df.index == '2018-12-31']

Last week of 2018:


city_id,C001,C002,C003,C004,C005,C007,C008,C009,C010,C011,...,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112
date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,11.8,14.0,27.1,1.9,26.0,20.2,5.2,18.4,16.8,2.0,...,2.3,28.5,6.4,21.7,12.7,23.0,1.4,27.9,28.1,20.2


In [28]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [29]:
# Dictionary to store predictions
predictions = {}

# Train a SARIMA model for each city using the historical data up to the end of 2018
for city in avg_temp_df.columns:
    # Filter training data up to the end of 2018
    train_data = avg_temp_df[avg_temp_df.index < '2019-01-01'][city]
    
    # Fit the SARIMA model
    model = SARIMAX(train_data, order=(5, 1, 0), seasonal_order=(1, 0, 1, 7))  # SARIMA parameters (p, d, q, P, D, Q, S)
    model_fit = model.fit(disp=False)
    
    # Forecast the average temperature for the first week of 2019
    forecast = model_fit.forecast(steps=7)
    
    # Store the forecast in the predictions dictionary
    predictions[city] = forecast.values

c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the 

In [30]:
# Create a DataFrame for the submission
submission = submission_key.copy()

# Function to retrieve predicted temperature for each row in submission_key
def get_predicted_temperature(row):
    city_id = row['city_id']
    date = pd.to_datetime(row['date'])
    days_since_start = (date - pd.to_datetime('2019-01-01')).days
    return predictions[city_id][days_since_start]

In [31]:
# Apply the function to create the 'avg_temp_c' column in submission
submission['avg_temp_c'] = submission.apply(get_predicted_temperature, axis=1)

# Use only submission_ID,avg_temp_c columns
submission = submission[['submission_ID', 'avg_temp_c']]

# Save the submission to a CSV file
submission.to_csv('./Data/submission_SARIMA.csv', index=False)

print("Predicted temperatures for submission:")
print(submission)

Predicted temperatures for submission:
     submission_ID  avg_temp_c
0                1   11.427853
1                2   10.849132
2                3   10.560276
3                4   10.231235
4                5   10.344537
..             ...         ...
695            696   21.158937
696            697   21.306203
697            698   21.111720
698            699   21.044020
699            700   21.139931

[700 rows x 2 columns]
